# **pyWadellRoundness**



In [10]:
%matplotlib  ipympl

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

from shapely.geometry import asPolygon, Point
from matplotlib import path  # Point in Polygon
from scipy import optimize
from scipy import signal
from scipy import spatial

# Functions

## Largest Inscribed Circle

In [11]:
def pyLIC(points):
    '''
    LIC: Largest Inner Circle between Points
    
    see: https://en.wikipedia.org/wiki/Largest_empty_sphere
    The largest inner circle problem is the problem of finding
    a circle of largest radius in the plane whose interior
    does not overlap with any given point and whose center
    is lying in the convex hull of the points.
    
    Hint: The LIC is centered at the internal Voronoi vertices
    with the largest shortest distance to the polygon

    Parameters
    ----------
    points : 2d numpy array with x, y coordinate of the points
             on the Concave Hull

    Returns
    -------
    LIC_Diameter: scalar
    '''

    # Step 1: get Voronoi vertices (scipy.spatial)
    vor_Verts = spatial.Voronoi(points).vertices

    # Step 2: select Voronoi vertices inside the convex hull (mpl)
    polygon = path.Path(points)
    selector = polygon.contains_points(vor_Verts)
    vor_Verts = vor_Verts[selector]  # Verts inside Polygon

    # Step 3: Get nearest neighbors as well as radius of LIC
    tree = spatial.KDTree(points)
    neighbores = tree.query(vor_Verts)
    idx = np.argmax(neighbores[0])
    Radius = neighbores[0][idx]
    Centroid = vor_Verts[idx]
    
    return Centroid[0], Centroid[1], Radius, 

## Smooth Boundary

In [12]:
def smooth_Boundary(concave_hull):
    """ Generate smooth boundary from concave hull,
    using scipy.signal.convolve
    Parameters
    ----------
    concave hull : obj
        List of coordinates of pixels of concave hull
    Returns
    -------
    boundary : 2D array
        List of coordinates of pixels in the boundary
    """
    
    concave_hull = np.array(concave_hull)

    # window size and type
    win_size = max(5, int(asPolygon(concave_hull).length / 15))
    win_size = min(10, win_size)
    win = signal.hamming(win_size)

    # extent boundary to solve some convolute topics
    ext_left = concave_hull[-win_size:]
    hull_extended = np.insert(concave_hull, 0, ext_left, axis=0)

    # apply filter
    X = signal.convolve(hull_extended[:, 0], win, mode='valid', method='auto') / sum(win)
    Y = signal.convolve(hull_extended[:, 1], win, mode='valid', method='auto') / sum(win)

    boundary = np.column_stack((X, Y))
    
    return boundary

## Fit Circle to Points

In [13]:
def _lsq_Circle(xy):
    """Find the least squares circle fitting a set of 2D points (x,y)
    see: http://www.scipy.org/Cookbook/Least_Squares_Circle
    
    Parameters
    ---------
    xy : Coordinates of the points as 2d np.array
         [(x1, y1), (x2, y2), ... (xn, yn)]
         
    Returns
    -------
    center : Center of circle (x, y)
    radius : Radius of circle (skalar)
    """
    
    def calc_Distance(xc, yc):
        """ calculate the distance of each 2D points from the center (xc, yc) """
        return np.sqrt((xs-xc)**2 + (ys-yc)**2)

    def cost_function(c):
        """ calculate the algebraic distance between the 2D points and
        the mean circle centered at c=(xc, yc) """
        Ri = calc_Distance(*c)
        return Ri - Ri.mean()
    
    # x, y coordinates of the points
    xs, ys = xy[:,0], xy[:,1]
    
    # just a good starting point
    p0 = np.mean(xs), np.mean(ys) # (=barycenter)
    
    try:
        # optimization: center of the lsq-Circle
        center, _ = optimize.leastsq(cost_function, p0)
        # radius of the lsq-Circle
        radius = calc_Distance(*center).mean()
        return center, radius
    
    except Exception as e:
        print('LSQ-Circle', e)

        return (np.nan, np.nan), np.nan

## Roundness after Wadell 1032

In [14]:
def _NumPoints(boundary_length):
    '''Number of points used in function Roundness to calculate radius
    of circles as a function of the length (number of points)
    of the boundary'''

    # empirical data
    bl  = [25, 50, 100]  # boundary length
    num = [ 4,  6,   8]  # number of points

    if boundary_length <= 25:
        num_points = min(num)
    elif boundary_length >= 100:
        num_points = max(num)
    else:
        f = sp.interpolate.interp1d(bl, num)
        num_points = f(boundary_length)
    return int(num_points)

def _WindowSize(boundary_length):
    '''windos size in points used in Roundness to find corners
    (valley), depending on the length (number of points) of
    the boundary'''

    # empirical data
    bl = [20, 50, 100, 200, 300, 400, 500]  # boundary length
    ws = [ 1,  2,   3,   9,  11,  13,  14]  # window width

    if boundary_length < 100:
        window_size = min(ws)
    elif boundary_length > 500:
        window_size = max(ws)
    else:
        f = sp.interpolate.interp1d(bl, ws)
        window_size = f(boundary_length)
    return int(window_size)

In [27]:
def Roundness_Wadell(boundary, LIC_R):

    # extent boundary to solve some 'wrap' topics
    num_points = _NumPoints(boundary.shape[0])
    ext_left = boundary[-num_points:]
    boundary = np.insert(boundary, 0, ext_left, axis=0)

    # radii of circles at all points of the boundary calculated with num_points
    radii, centers = [], []
    for i in range(len(boundary)-num_points):
        points = boundary[i:i+num_points]
        c_i, r_i = _lsq_Circle(points)
        centers.append(c_i)
        radii.append(r_i)     
    centers = np.array(centers) # list to np.array
    radii   = np.array(radii)

    # find valleys (corners, radius at P smaller as radius of Points within win_size)
    win_size = _WindowSize(boundary.shape[0])

    selector = signal.argrelextrema(radii, comparator=np.less, order=win_size)
    centers = centers[selector]
    radii   = radii[selector]

    # Cirles with radius < radius of largest inner circle
    selector = []
    for idx, r in enumerate(radii):
        if radii[idx] < LIC_R: selector.append(idx)
    radii   = radii[selector]
    centers = centers[selector]

    # Circles within boundary
    selector = []
    for idx in range(len(radii)):

        # points (shapely) on circle
        n = 10
        t = np.linspace(0, 2*np.pi, n, endpoint=True)
        x = radii[idx] * np.cos(t) + centers[idx][0]
        y = radii[idx] * np.sin(t) + centers[idx][1]
        points_ofCircle = np.stack((x, y), axis=-1)
        points_ofCircle = [Point(p) for p in points_ofCircle]

        # points within boundary
        polygon =asPolygon(boundary).buffer(1)
        within = [p.within(polygon) for p in points_ofCircle]  # True!
        selector.append(all(within))

    radii   = radii[selector]
    centers = centers[selector]
    R = np.mean(radii) / LIC_R

    return R, np.array(radii), np.array(centers)


# Test
## Prepare data

In [28]:
# read test Data
points = pd.read_csv('test_dat.csv').to_numpy()

# smoothed boundary
boundary = smooth_Boundary(points)

# largest Inscribed Circle
LIC_x, LIC_y, LIC_R = pyLIC(points)

## Test Function

In [29]:
# Wadell Corners: R, radii und centers of circles defined by corners
R, Wadell_radii, Wadell_centers = Roundness_Wadell(boundary, LIC_R)

# Show Result

In [30]:
fig, ax = plt.subplots()
ax.set_aspect('equal')

ax.set_title('Roundness Wadell: ' + str(round(R, 2)))

ax.plot(points[:, 0], points[:, 1], 'k.', label='original data')

ax.plot(boundary[:, 0], boundary[:, 1], 'b-', lw=1, label='smoothed boundary')
circle = plt.Circle(xy=(LIC_x, LIC_y), radius=LIC_R, 
                    ec='r', fc='None', lw=2)
ax.add_artist(circle)

for idx, _ in enumerate(Wadell_radii):
    circle = plt.Circle(xy=Wadell_centers[idx], radius=Wadell_radii[idx],
                        ec='g', color='None', lw=1.5)
    ax.add_artist(circle)




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …